In [1]:
from torch import cuda, float16
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig, pipeline

DEVICE = f"cuda:{cuda.current_device()}" if cuda.is_available() else "cpu"
print(f"Current device is: {DEVICE}")

Current device is: cuda:0


In [3]:
# MODEL_ID = "deepseek-ai/DeepSeek-R1-Distill-Qwen-1.5B"
# MODEL_ID = "deepseek-ai/deepseek-coder-7b-instruct"
MODEL_ID = "unsloth/Qwen2.5-3B-unsloth-bnb-4bit"

# Квантуем в 4 бита, чтобы поместилось в VRAM 6–8 ГБ
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,                          # включить 4-битное квантование
    bnb_4bit_quant_type="nf4",                  # тип квантования: "nf4" (Normalized Float 4) или "fp4"
    bnb_4bit_use_double_quant=True,             # включить двойное квантование (дополнительная компрессия)
    bnb_4bit_compute_dtype=float16        # тип данных для вычислений (например, bfloat16 (недоступен на T4), float16)
)

print("Загружаем модель …")
model = AutoModelForCausalLM.from_pretrained(
    MODEL_ID,
    quantization_config=bnb_config,
    device_map="auto",
    trust_remote_code=True,
)

tokenizer = AutoTokenizer.from_pretrained(MODEL_ID)

llm_pipeline = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    torch_dtype=float16,
)

Загружаем модель …


config.json: 0.00B [00:00, ?B/s]

c:\Users\Anastasiya Fedotova\Desktop\DS&ML\github\rag_testing\.venv\Lib\site-packages\huggingface_hub\file_download.py:143: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\Anastasiya Fedotova\.cache\huggingface\hub\models--unsloth--Qwen2.5-3B-unsloth-bnb-4bit. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


model.safetensors:   0%|          | 0.00/2.56G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/171 [00:00<?, ?B/s]

...

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json:   0%|          | 0.00/11.4M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/605 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/617 [00:00<?, ?B/s]

Device set to use cuda:0


In [4]:
def ask_DeepSeek(prompt: str):
    resp = llm_pipeline(
        prompt,
        max_new_tokens=200,     # генерируем ≤ 200 новых токенов
        do_sample=True,         # сэмплирование позволяет модели додумывать
        truncation=True,        # обрываем слишком длинные ответы
        top_k=20,               # топ-20 наиболее вероятных токенов на выходе генерации
        num_return_sequences=1, # 1 вариант ответа
        eos_token_id=tokenizer.eos_token_id, # что считать концом последовательности
    )[0]["generated_text"]
    print(resp)

ask_DeepSeek("Объясни, пожалуйста, что такое ‘State of the Union’. Ответь одним абзацем на русском.")

Объясни, пожалуйста, что такое ‘State of the Union’. Ответь одним абзацем на русском. В английском языке это выражение означает «Социальный статус», «Состояние здоровья», «Состояние общества». Т.е. оно может быть использовано в широком смысле: об общем «уровне» общественного развития, социальной жизни, политического развития страны. Также оно может употребляться в более конкретном и специфическом смысле, чтобы охарактеризовать конкретный «уровень» (степень) развития какой-либо «сферы» общественной жизни, например, экономики или медицины. В данном случае «State of the Union» употребляется в более широком смысле и говорит о том, что президент США дает доклад об общем «уровне» экономического и социального развития страны.


In [18]:
from datasets import load_dataset
from langchain.docstore.document import Document

wiki_ds = load_dataset("Den4ikAI/russian_cleared_wikipedia", split="train")

# Document
corpus_docs = [
    Document(page_content=rec["sample"])
    for rec in wiki_ds
]

print("Документов:", len(corpus_docs))
print(corpus_docs[0].page_content[:200], "…")

Документов: 6284
Ева  — в авраамических религиях — праматерь всех людей, первая женщина, жена Адама, созданная из его ребра, мать Каина, Авеля и Сифа.Библейский рассказ о сотворении Адама и Евы, грехопадении и изгнани …


In [19]:
# from langchain.document_loaders import TextLoader  # загружает текстовые файлы и превращает их в объекты Document для LangChain.
from langchain.text_splitter import RecursiveCharacterTextSplitter  # рекурсивно разбивает длинный текст на более мелкие фрагменты (chunks).
from langchain_huggingface import HuggingFaceEmbeddings  # оборачивает модели из HuggingFace для получения эмбеддингов текста.
from langchain.vectorstores import Chroma  # векторное хранилище Chroma: сохраняет и ищет эмбеддинги.

from langchain_huggingface import HuggingFacePipeline  # использует HuggingFace Transformers pipeline как LLM-модуль в LangChain.
from langchain.chains import RetrievalQA  # готовая цепочка «поиск + генерация ответа» (Retrieval-augmented QA).

In [20]:
splitter = RecursiveCharacterTextSplitter(
    chunk_size=512, chunk_overlap=50
)

docs = splitter.split_documents(corpus_docs)
print("Чанков:", len(docs))

Чанков: 67536


In [26]:
embeddings = HuggingFaceEmbeddings(
    # model_name="sentence-transformers/all-mpnet-base-v2",
    model_name="intfloat/multilingual-e5-base", #Russian semantic search
    model_kwargs={"device": "cuda"},
)

vectordb = Chroma.from_documents(
    documents=docs,               # либо corpus_docs, если без сплиттера
    embedding=embeddings,
    persist_directory="chroma_ragmini"  # директория для хранения векторной базы
)
vectordb.persist() # в нашем рабочем пространстве создалась директория - векторное хранилище

modules.json:   0%|          | 0.00/387 [00:00<?, ?B/s]

c:\Users\Anastasiya Fedotova\Desktop\DS&ML\github\rag_testing\.venv\Lib\site-packages\huggingface_hub\file_download.py:143: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\Anastasiya Fedotova\.cache\huggingface\hub\models--intfloat--multilingual-e5-base. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/57.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/694 [00:00<?, ?B/s]

...

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.1M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/280 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/200 [00:00<?, ?B/s]

Failed to send telemetry event ClientStartEvent: capture() takes 1 positional argument but 3 were given
Failed to send telemetry event ClientCreateCollectionEvent: capture() takes 1 positional argument but 3 were given
C:\Users\Anastasiya Fedotova\AppData\Local\Temp\ipykernel_26980\3103602615.py:12: LangChainDeprecationWarning: Since Chroma 0.4.x the manual persistence method is no longer supported as docs are automatically persisted.
  vectordb.persist() # в нашем рабочем пространстве создалась директория - векторное хранилище


In [27]:
from transformers import GenerationConfig

gen_cfg = GenerationConfig.from_pretrained(MODEL_ID)
gen_cfg.max_length     = 2048     # допустим любой контекст до 2048 токенов
gen_cfg.max_new_tokens = 128      # ограничиваем длину генерируемого ответа
# gen_cfg.do_sample = True
# gen_cfg.temperature = 0.0
# gen_cfg.top_p = 0.9

model.generation_config = gen_cfg   # привязываем к модели

In [28]:
from langchain import PromptTemplate


simple_custom_prompt = PromptTemplate(
    template="""<think>
Use the following context to answer the question. Be precise and short. Avoid repetitions. Do not explain the answer unless asked to do so.

Context: {context}
Question: {question}

First think logically, then give me the answer.
</think>
<answer>""",
    input_variables=["context", "question"]
)


simple_custom_prompt = PromptTemplate(
    template="""<think>
Сначала подумай, потом ответь. Избегай повторений и будь краток, если не попросят пояснений.

Context: {context}
Question: {question}

</think>
<answer>""",
    input_variables=["context", "question"]
)

llm = HuggingFacePipeline(
    pipeline=llm_pipeline,          # тот, что мы собрали для DeepSeek
    model_kwargs={
        "max_new_tokens": 128,      # генерируем 128 новых токенов
        "temperature": 0.1,         # опционально
        "do_sample": False          # чтобы ответ был детерминирован
    }
)

retriever = vectordb.as_retriever()

qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=retriever,
    chain_type_kwargs={"prompt": simple_custom_prompt},  # ваш кастомный промпт
    return_source_documents=False,
)

In [32]:
question = "Расскажи сказку про рыбака и рыбку"

answer_str = qa_chain.run(question)
marker = "</think>"
print("Ответ:", answer_str.split(marker)[1].strip())

Ответ: <answer>Сказка о рыбке и рыбаке рассказывает следующую историю: Однажды рыбак ловил рыбу на реке, и рыба попала в его ловушку. Рыбка была очень красивая и умная. Она знала, что рыбак собирался уйти, и решила развлечь его и остановить. Рыбка начала петь и плясать для рыбака, и рыбак посмеялся над ней и решил заставить ее работать. Он сказал рыбке, что если она не будет ловиться, он ее отпустит. Рыбка согласилась на это и стала упорно пытаться ловить рыбу. В результате рыбак пришлось отпустить рыбку, и она ушла в воду. Рыбка пошла на берег и сказала рыбаку, что он должен было ей быть благодарен, потому что она помогла ему почувствовать свою силу и упорство. Сказка о рыбке и рыбаке учит нас, что иногда нужно быть усердным и не бросаться в переделки, а если необходимо, то можно быть умным и убедить другого человека, а не заставить того работать, чтобы добраться до него.


In [33]:
# question = "Did Lincoln sign the National Banking Act of 1863? First say yes or no, then justify shortly."

# answer_str = qa_chain.run(question)
# print("Ответ:",  answer_str.split(marker)[1].strip())

In [49]:
qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=vectordb.as_retriever(search_kwargs={"k": 1}), # здесь мы подаем ограничение на кол-во возвращаемых док-ов
    chain_type_kwargs={"prompt": simple_custom_prompt},
    return_source_documents=True,
)

# попробуем другую функцию - она возвращает и использованные документы
response = qa_chain.invoke("Коротко расскажи про разницу яблок антоновка и яблок сорта белый налив")
print("Ответ:", response["result"].split(marker, 1)[1].strip())
print("\nSource documents:")
for doc in response["source_documents"]:
    print(doc.page_content[:80], "…")

Ответ: <answer>Яблоки антоновского сорта и белый налив - это два разных сорта яблок. Яблоки антоновского сорта имеют более крупные размеры и ярко-красный цвет с белыми точками на кожуре. Они богаты витамином С и содержат низкую калорийность, что делает их идеальным выбором для тех, кто следит за своим весом или диетой. Белый налив, напротив, имеет более маленькие размеры и имеет более мягкий мякоть. Он также богат витамином С и содержит низкую калорийность, но его кислотность более насыщена, чем у антоновского сорта.</answer>

Source documents:
в тесте, приготовляют начинки для пирогов, тортов и пирожных, очень популярны яб …


In [50]:
response["source_documents"]

[Document(metadata={}, page_content='в тесте, приготовляют начинки для пирогов, тортов и пирожных, очень популярны яблочные пироги.Сушёные яблоки являются хорошим источником легкоусваиваемых сахаров, микроэлементов, а в семенах одного среднего плода содержится о о суточной нормы йода.Например, яблоки антоновского сорта в 100 граммах при калорийности в 48 ккал содержат: 0,3 г белков, 11,5 г углеводов, 0,02\xa0мг витамина B1, 4,9\xa0мг витамина С, 16 мг кальция и 86 мг калия.Виды:Ещё более 200 видовых названий этого рода имеют в The Plant List статус')]

In [82]:
# import json, tqdm

# # функция для удаления всего до и включая маркер </think>
# def strip_cot(raw: str, marker: str = "</think>") -> str:
#     parts = raw.split(marker, 1)
#     # если не влез COT в токены, выводим ответ как есть
#     return parts[1].strip() if len(parts) == 2 else raw.strip()

# testset = load_dataset("rag-datasets/rag-mini-wikipedia", "question-answer")["test"]

# total, correct = 0, 0

# print("Проверка 10 примеров:\n")
# for sample in tqdm.tqdm(testset.select(range(4, 14))):
#     q, gold = sample["question"], sample["answer"]

#     result_dict = qa_chain.invoke(q)
#     raw = result_dict["result"]
#     # обрезаем COT
#     pred = strip_cot(raw)

#     print(f"\nВопрос: {q}")
#     print(f"Ожидаемые ответы: {gold}")
#     print(f"Ответ модели: {pred}")

#     if any(g.lower() in pred for g in gold.lower()):
#         correct += 1
#     total += 1

# print(f"\nAccuracy@10: {correct/total:.2%}")

In [14]:
# print(f"\nAccuracy@10: {correct/total:.2%}")

In [16]:
# model.name_or_path

In [17]:
import os

# Стандартный путь кэша
cache_path = os.path.expanduser('~/.cache/huggingface/hub/')
print(f"📁 Кэш моделей: {cache_path}")

# # Ищем конкретную модель
# model_folder = f"models--{model_name.replace('/', '--')}"
# full_path = os.path.join(cache_path, model_folder)
# print(f"📁 Папка модели: {full_path}")

# # Проверяем существует ли
# if os.path.exists(full_path):
#     print("✅ Модель найдена!")
#     # Покажем что внутри
#     for item in os.listdir(full_path):
#         print(f"  📂 {item}")
# else:
#     print("❌ Модель не найдена в кэше")

📁 Кэш моделей: C:\Users\Anastasiya Fedotova/.cache/huggingface/hub/
